<a href="https://colab.research.google.com/github/diianaesc/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version 

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [910 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-22 02:24:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.93MB/s    in 0.2s    

2022-08-22 02:24:42 (5.93 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Vine-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in the Review dataset as a DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
books_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)
books_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [ ]:
# Drop null values
dropna_df = books_df.dropna()
dropna_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [ ]:
# Drop duplicates
clean_df = dropna_df.drop_duplicates()
clean_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   53055712| R1002FAEXYJTB|0061002860|     121809515|The Murder of Rog...|           Books|          4|            0|          0|   N|                N|   Suspect everyone!|I am so glad our ...|1998-11-07 00:00:00|
|         US|   52453598|R100G0ZBG3OND5|0380781484|     592019319|             Captive| 

In [ ]:
from pyspark.sql.functions import to_date

In [ ]:
# Create the vine_table. DataFrame
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R1002FAEXYJTB|          4|            0|          0|   N|                N|
|R100G0ZBG3OND5|          5|            4|          6|   N|                N|
|R100I4QUZNIIR6|          5|            9|         15|   N|                N|
|R101J95JQTCU0G|          5|            4|          6|   N|                N|
|R101JCRAXEZ2XN|          2|            0|          7|   N|                N|
|R101LMD5SARW1G|          5|            3|          4|   N|                N|
|R102B6ZV3IX49B|          4|           10|         15|   N|                Y|
|R102HJVFYRC897|          5|            0|          0|   N|                N|
|R102IB6OP9TWGG|          5|            1|          1|   N|                N|
|R102LF0BEMGIW2|          5|           31|         32|   N|     

Deliverable 2: Determine Bias of Vine Reviews

In [ ]:
# Filter the Vine DataFrame by those that had 20+ total votes
df2 = vine_df.filter(vine_df["total_votes"] >= 20)
df2.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R102LF0BEMGIW2|          5|           31|         32|   N|                N|
|R104G2EZ1941MK|          4|           22|         24|   N|                N|
|R104M8GQWNMKXX|          5|           29|         32|   N|                Y|
|R1053OFVTG77Y8|          5|           20|         21|   N|                N|
|R1083NX8PQZTYM|          4|           19|         21|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filter the new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
df3 = df2.filter(df2["helpful_votes"]/df2["total_votes"] >= 0.5)
df3.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R102LF0BEMGIW2|          5|           31|         32|   N|                N|
|R104G2EZ1941MK|          4|           22|         24|   N|                N|
|R104M8GQWNMKXX|          5|           29|         32|   N|                Y|
|R1053OFVTG77Y8|          5|           20|         21|   N|                N|
|R1083NX8PQZTYM|          4|           19|         21|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
df3.describe().show()

+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|      total_votes|  vine|verified_purchase|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|  count|        403778|            403778|            403778|           403778|403778|           403778|
|   mean|          null|4.0784639083853005|41.521259206791854| 47.5853340201794|  null|             null|
| stddev|          null|1.3710364012325926|  62.7092964444989|68.00266620962232|  null|             null|
|    min|R1001GCNUFG47T|                 1|                10|               20|     N|                N|
|    max| RZZZOC01ZG3XG|                 5|             15835|            16352|     N|                Y|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+



In [ ]:
# Filter the DataFrame to retrieve all the rows where a review was written as part of the Vine program, vine == 'Y'
vine_yes_df = df3.filter(df3["vine"] == "Y")
vine_yes_df.show(20)

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [ ]:
# Filter the Vine DataFrame by those that *do not* have a vine review
vine_no_df = df3.filter(df3["vine"] == "N")
vine_no_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R102LF0BEMGIW2|          5|           31|         32|   N|                N|
|R104G2EZ1941MK|          4|           22|         24|   N|                N|
|R104M8GQWNMKXX|          5|           29|         32|   N|                Y|
|R1053OFVTG77Y8|          5|           20|         21|   N|                N|
|R1083NX8PQZTYM|          4|           19|         21|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import col, avg

In [ ]:
# Get summary statistics of those that had vine reviews
vine_yes_df.describe().show()

+-------+---------+-----------+-------------+-----------+----+-----------------+
|summary|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-------+---------+-----------+-------------+-----------+----+-----------------+
|  count|        0|          0|            0|          0|   0|                0|
|   mean|     null|       null|         null|       null|null|             null|
| stddev|     null|       null|         null|       null|null|             null|
|    min|     null|       null|         null|       null|null|             null|
|    max|     null|       null|         null|       null|null|             null|
+-------+---------+-----------+-------------+-----------+----+-----------------+



In [ ]:
# Get summary statistics of those that did not have vine reviews
vine_no_df.describe().show()

+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|      total_votes|  vine|verified_purchase|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+
|  count|        403778|            403778|            403778|           403778|403778|           403778|
|   mean|          null|4.0784639083853005|41.521259206791854| 47.5853340201794|  null|             null|
| stddev|          null|1.3710364012325926|  62.7092964444989|68.00266620962232|  null|             null|
|    min|R1001GCNUFG47T|                 1|                10|               20|     N|                N|
|    max| RZZZOC01ZG3XG|                 5|             15835|            16352|     N|                Y|
+-------+--------------+------------------+------------------+-----------------+------+-----------------+



In [ ]:
# Percentage of 5-star rating that had vine reviews
paid_five_star_number = vine_yes_df[vine_yes_df['star_rating']== 5].count()
paid_number = vine_yes_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

ZeroDivisionError: ignored

In [ ]:
# Get percentage of 5-star rating among those record that did not have vine reviews
paid_five_star_number = vine_no_df[vine_no_df['star_rating']== 5].count()
paid_number = vine_no_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

403778
242868
0.6014889369901283
